In [35]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [36]:
df=pd.read_csv("./mnist.csv")

In [37]:
feature = df[df.columns[:-1]]
target = df[df.columns[-1]]

In [38]:
target=pd.get_dummies(target).astype("int8")
target

,0,1,2,3,4,5,6,7,8,9
0,0,0,0,0,0,1,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
69995,0,0,1,0,0,0,0,0,0,0
69996,0,0,0,1,0,0,0,0,0,0
69997,0,0,0,0,1,0,0,0,0,0
69998,0,0,0,0,0,1,0,0,0,0


In [39]:
x_train, x_test, y_train, y_test = train_test_split(feature, target, test_size=1/7, random_state=42, stratify=target)

In [40]:
x_train = torch.tensor(x_train.values, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.long)
x_test = torch.tensor(x_test.values, dtype=torch.float32)
y_test = torch.tensor(y_test.values, dtype=torch.long)

In [41]:
# 데이터셋 생성
import torch.utils.data.dataloader as dataloader
import torch.utils.data.dataset as dataset

class MyDataset(dataset.Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __len__(self):
        return len(self.x)
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [42]:
dataset_train = MyDataset(x_train, y_train)
dataset_test = MyDataset(x_test, y_test)

In [43]:
class model(nn.Module): # 가중치 초기화 
    def __init__(self):
        super(model, self).__init__()
        self.fc1 = nn.Linear(784, 300)
        self.fc2 = nn.Linear(300, 10)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [44]:
model2 = model()
optimizer = optim.Adam(model2.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

train_loader = dataloader.DataLoader(dataset_train, batch_size=100, shuffle=True)
test_loader = dataloader.DataLoader(dataset_test, batch_size=100, shuffle=False)

In [45]:
from torchmetrics.functional.classification import accuracy, f1_score
model2.train()

for epoch in range(15):
    
    for x,y in train_loader:
        output = model2(x)
        loss = criterion(output, np.argmax(y, axis=1))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        with torch.no_grad():
            #print(f"loss: {loss.item()}")
            pass
        
        
        

    print(f"================================={epoch}=================================")
    output = model2(x_train)
    acc = accuracy(output, np.argmax(y_train, axis=1), task="multiclass", num_classes=10, average="macro")
    f1 = f1_score(output, np.argmax(y_train, axis=1), task="multiclass", num_classes=10, average="macro")
    print(f"acc : {acc}, f1 : {f1}")
    
    
    output = model2(x_test)
    print(f"=================================test score=================================")
    print(accuracy(output, np.argmax(y_test, axis=1), task="multiclass", num_classes=10, average="macro"))
    print(f1_score(output, np.argmax(y_test, axis=1), task="multiclass", num_classes=10, average="macro"))

    if np.abs(f1-1) < 0.000001:
        print("발동!")
        torch.save(model2,"./model.pth")
        break

=================================0=================================
acc : 0.9680914878845215, f1 : 0.9680865406990051
=================================test score=================================
tensor(0.9600)
tensor(0.9600)
=================================1=================================
acc : 0.973896324634552, f1 : 0.9738903045654297
=================================test score=================================
tensor(0.9634)
tensor(0.9633)
=================================2=================================
acc : 0.9802483916282654, f1 : 0.9803823232650757
=================================test score=================================
tensor(0.9681)
tensor(0.9683)
=================================3=================================
acc : 0.9762303233146667, f1 : 0.9762227535247803
=================================test score=================================
tensor(0.9658)
tensor(0.9658)
=================================4=================================
acc : 0.9841017723083496, f1 : 0.

In [47]:
model2=torch.load("./model.pth")
model2.eval()


output = model2(x_test)
print(f"=================================test score=================================")
print(accuracy(output, np.argmax(y_test, axis=1), task="multiclass", num_classes=10, average="macro"))
print(f1_score(output, np.argmax(y_test, axis=1), task="multiclass", num_classes=10, average="macro"))

=================================test score=================================
tensor(0.9943)
tensor(0.9943)
